In [8]:
from pathlib import Path
from datetime import date

import geopandas as gdp

from cropharvest.datasets import CropHarvest, CropHarvestLabels
from cropharvest.engineer import Engineer
from cropharvest.bands import BANDS, DYNAMIC_BANDS, STATIC_BANDS, REMOVED_BANDS
from cropharvest.columns import RequiredColumns, NullableColumns

In [9]:
data_root = Path('../data/features/nigeria-cropharvest')
file_name = 'labels.geojson'

In [10]:
gdf = gdp.read_file('../data/processed/nigeria/data.geojson')
start_date = date(2019, 4, 3)
end_date = date(2020, 3, 28)
shapefile = gdp.read_file('../data/features/nigeria-cropharvest/nigeria_stratified_labelled_v1_splits_resampled.shp')
shapefile['identifier'] = shapefile['identifier'].astype(int)
gdf = gdf.merge(shapefile[['identifier', 'new_set']], left_on='index', right_on='identifier', suffixes=(None, '_y'))

#gdf['export_identifier'] = gdf.apply(lambda row: f"{str(row.loc['index'])}_{str(row.loc['start_date'])}_{str(row.loc['end_date'])}", axis=1)
gdf['start_date'] = start_date # required by EarthEngineExporter
gdf['end_date'] = end_date # required by EarthEngineExporter
gdf['export_end_date'] = end_date # required by Engineer
gdf['dataset'] = 'nigeria'
gdf['export_identifier'] = gdf.apply(lambda row: f"{str(row.loc['index'])}_{str(row.loc['dataset'])}_{str(row.loc['start_date'])}_{str(row.loc['end_date'])}", axis=1)
gdf['is_test'] = gdf['new_set'] == 'testing'
gdf['label'] = None
gdf

,is_crop,lat,lon,index,geometry,identifier,new_set,start_date,end_date,export_end_date,dataset,export_identifier,is_test,label
0,0,13.062977,7.169715,0,POINT (7.16971 13.06298),0,validation,2019-04-03,2020-03-28,2020-03-28,nigeria,0_nigeria_2019-04-03_2020-03-28,False,None
1,0,9.861582,5.029259,1,POINT (5.02926 9.86158),1,training,2019-04-03,2020-03-28,2020-03-28,nigeria,1_nigeria_2019-04-03_2020-03-28,False,None
2,0,10.442239,11.673885,2,POINT (11.67388 10.44224),2,testing,2019-04-03,2020-03-28,2020-03-28,nigeria,2_nigeria_2019-04-03_2020-03-28,True,None
3,0,6.506252,6.584187,3,POINT (6.58419 6.50625),3,training,2019-04-03,2020-03-28,2020-03-28,nigeria,3_nigeria_2019-04-03_2020-03-28,False,None
4,0,7.980531,7.099293,4,POINT (7.09929 7.98053),4,testing,2019-04-03,2020-03-28,2020-03-28,nigeria,4_nigeria_2019-04-03_2020-03-28,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,0,11.505429,6.910431,1822,POINT (6.91043 11.50543),1822,validation,2019-04-03,2020-03-28,2020-03-28,nigeria,1822_nigeria_2019-04-03_2020-03-28,False,None
1823,0,13.281881,13.412151,1823,POINT (13.41215 13.28188),1823,validation,2019-04-03,2020-03-28,2020-03-28,nigeria,1823_nigeria_2019-04-03_2020-03-28,False,None
1824,0,8.912081,3.703954,1824,POINT (3.70395 8.91208),1824,testing,2019-04-03,2020-03-28,2020-03-28,nigeria,1824_nigeria_2019-04-03_2020-03-28,True,None
1825,1,8.653739,4.971758,1825,POINT (4.97176 8.65374),1825,validation,2019-04-03,2020-03-28,2020-03-28,nigeria,1825_nigeria_2019-04-03_2020-03-28,False,None


In [11]:
# Save labels 
gdf['start_date'] = gdf['start_date'].astype(str)
gdf['end_date'] = gdf['end_date'].astype(str)
gdf['export_end_date'] = gdf['export_end_date'].astype(str)
gdf.to_file(data_root / file_name, driver='GeoJSON')

In [12]:
engineer = Engineer(data_root)

In [13]:
## Modify file name so that Engineer.process_filename() works
# folder = Path('data/nigeria/eo_data/')
# files = list(folder.glob('*.tif')) 
# for file_path in files:
#     parts = file_path.name.split('_')
#     new_file_name = '_'.join([str(parts[0]) + '-' + 'nigeria, *parts[1:]])
#     file_path.rename(folder / new_file_name)

In [15]:
engineer.create_h5_dataset()

  0%|          | 0/1827 [00:00<?, ?it/s]

/home/gajo/miniconda3/envs/nigeria-crop-mask-gpu/lib/python3.7/site-packages/cropharvest/engineer.py:348: RuntimeWarning: Mean of empty slice
  mean_per_band = np.nanmean(array, axis=0)
 27%|██▋       | 487/1827 [00:00<00:00, 1598.96it/s]/home/gajo/miniconda3/envs/nigeria-crop-mask-gpu/lib/python3.7/site-packages/cropharvest/engineer.py:348: RuntimeWarning: Mean of empty slice
  mean_per_band = np.nanmean(array, axis=0)
 35%|███▌      | 647/1827 [00:00<00:00, 1294.52it/s]/home/gajo/miniconda3/envs/nigeria-crop-mask-gpu/lib/python3.7/site-packages/cropharvest/engineer.py:348: RuntimeWarning: Mean of empty slice
  mean_per_band = np.nanmean(array, axis=0)
 42%|████▏     | 768/1827 [00:00<00:01, 1020.63it/s]/home/gajo/miniconda3/envs/nigeria-crop-mask-gpu/lib/python3.7/site-packages/cropharvest/engineer.py:348: RuntimeWarning: Mean of empty slice
  mean_per_band = np.nanmean(array, axis=0)
 47%|████▋     | 863/1827 [00:00<00:01, 836.61it/s] /home/gajo/miniconda3/envs/nigeria-crop-mask-gpu

Wrote 0 files, skipped 5 files
No normalizing dict calculated! Make sure to call update_normalizing_values
No normalizing dict calculated!
